In [1]:
# Install libraries if not available
!pip install tensorflow keras sklearn nltk

# Import
import pandas as pd
import numpy as np
import re
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

nltk.download('stopwords')


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
!wget https://raw.githubusercontent.com/justmarkham/DAT8/master/data/sms.tsv


--2025-12-10 04:40:40--  https://raw.githubusercontent.com/justmarkham/DAT8/master/data/sms.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 477907 (467K) [text/plain]
Saving to: ‘sms.tsv’

sms.tsv             100%[===================>] 466.71K  --.-KB/s    in 0.04s   

2025-12-10 04:40:40 (12.9 MB/s) - ‘sms.tsv’ saved [477907/477907]



In [4]:
import pandas as pd

df = pd.read_csv("sms.tsv", sep='\t', header=None, names=['label','message'])
df.head()


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
#preprocess dataset
# Convert label to 0/1
df['label'] = df['label'].map({'ham':0, 'spam':1})

# Clean text
def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'http\S+', '', text)  # remove links
    text = re.sub(r'[^a-zA-Z ]', '', text)  # remove punctuation/numbers
    return text

df['clean_message'] = df['message'].apply(clean_text)
df.head()


,label,message,clean_message
0,0,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,0,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in a wkly comp to win fa cup final...
3,0,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...


In [8]:
#tokenize the text
tokenizer = Tokenizer(num_words=5000)  # keep top 5000 words
tokenizer.fit_on_texts(df['clean_message'])

X = tokenizer.texts_to_sequences(df['clean_message'])
X = pad_sequences(X, maxlen=50)  # pad sequences to length 50

y = df['label'].values


In [9]:
#train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
#buid LSTM MODEL
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=50))
model.add(LSTM(64))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
#train
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)


Epoch 1/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 91ms/step - accuracy: 0.8499 - loss: 0.4286 - val_accuracy: 0.9596 - val_loss: 0.1503
Epoch 2/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - accuracy: 0.9753 - loss: 0.1047 - val_accuracy: 0.9776 - val_loss: 0.0770
Epoch 3/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.9897 - loss: 0.0361 - val_accuracy: 0.9787 - val_loss: 0.0737
Epoch 4/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - accuracy: 0.9968 - loss: 0.0161 - val_accuracy: 0.9809 - val_loss: 0.0718
Epoch 5/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - accuracy: 0.9979 - loss: 0.0107 - val_accuracy: 0.9809 - val_loss: 0.0912


In [12]:
#evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)



35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9870 - loss: 0.0595
Test Accuracy: 0.9883407950401306


In [13]:
#predict new messages
def predict_message(msg):
    seq = tokenizer.texts_to_sequences([clean_text(msg)])
    padded = pad_sequences(seq, maxlen=50)
    pred = model.predict(padded)[0][0]
    return "SCAM" if pred > 0.5 else "SAFE"

predict_message("Congratulations! You won a free iPhone. Click here!")
predict_message("Hey, are we meeting today?")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 430ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


'SAFE'

In [14]:
# Example
test_msgs = [
    "Hey, are we meeting at 5 PM today?",
    "Congratulations! You won ₹1,00,000. Click here to claim.",
    "Lunch at the new cafe today?",
    "Get a free iPhone now, limited time offer!"
]

for msg in test_msgs:
    print(msg, " --> ", predict_message(msg))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Hey, are we meeting at 5 PM today?  -->  SAFE
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Congratulations! You won ₹1,00,000. Click here to claim.  -->  SCAM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Lunch at the new cafe today?  -->  SAFE
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Get a free iPhone now, limited time offer!  -->  SAFE


# WhatsApp Scam Detector: AI-Based Fraud Message Classifier

## Project Overview
This project is an AI-powered **WhatsApp Scam Detector** that identifies whether a WhatsApp/SMS message is **safe (ham)** or **scam/fraud (spam)**.  
It uses **Natural Language Processing (NLP)** and a **Deep Learning LSTM model** to classify messages.  
This project is beginner-friendly and fully implemented in **Google Colab**.

---

## Features
- Classifies messages as **SAFE** or **SCAM**
- Uses **LSTM Deep Learning** for text classification
- Clean and preprocess WhatsApp/SMS messages
- Easy to test with **your own messages**
- Can be extended to a web app using **Streamlit / Gradio**

---

## Dataset
The project uses the **SMS Spam Collection Dataset**:  
- Format: Tab-separated file (`.tsv`)  
- Columns: `label` (ham/spam) and `message` (text)  
- Source: [SMS Spam Collection](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/sms.tsv)  

---

## Step-by-Step Guide

### 1. Install & Import Libraries
```python
!pip install tensorflow keras sklearn nltk

import pandas as pd
import numpy as np
import re
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
